In [1]:
pip install pandas numpy matplotlib seaborn statsmodels linearmodels

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

# Load the dataset (make sure the CSV is in the same folder as your notebook)
df = pd.read_csv("Dataset2.csv")

# Inspect the first 5 rows
print(df.head())

# Check shape (rows, columns)
print(df.shape)

# Quick info on columns & datatypes
print(df.info())9870
    \321`

  stock     endDate  accountsPayable    inventory  longTermDebt  \
0   IVC  2019-12-31       88003000.0  121184000.0   219464000.0   
1   IVC  2018-12-31       92469000.0  128644000.0   225733000.0   
2   IVC  2017-12-31       90566000.0  121933000.0   241405000.0   
3   IVC  2016-12-31       88236000.0  135644000.0   146088000.0   
4  CSLT  2019-12-31       19596000.0          NaN           NaN   

   netReceivables  netTangibleAssets  longTermInvestments  totalCurrentAssets  \
0     140013000.0        -91334000.0              85000.0         355877000.0   
1     138476000.0        -48632000.0              90000.0         397410000.0   
2     143464000.0         -8233000.0             103000.0         456914000.0   
3     132707000.0         32762000.0             108000.0         409072000.0   
4      31397000.0         68911000.0                  NaN          95470000.0   

   propertyPlantEquipment  ...   totalAssets  otherCurrentLiab  \
0              92183000.0  ...  8.521260e+08

In [4]:
import numpy as np

# Define a safe division function
def safe_div(a, b):
    return np.where(b == 0, np.nan, a / b)

# Create ratios
df['leverage'] = safe_div(df['totalLiab'], df['totalAssets'])
df['equity_to_assets'] = safe_div(df['totalStockholderEquity'], df['totalAssets'])
df['current_ratio'] = safe_div(df['totalCurrentAssets'], df['totalCurrentLiabilities'])
df['quick_ratio'] = safe_div(df['cash'].fillna(0) + df['netReceivables'].fillna(0), df['totalCurrentLiabilities'])
df['wc_to_assets'] = safe_div(df['totalCurrentAssets'] - df['totalCurrentLiabilities'], df['totalAssets'])
df['cash_to_assets'] = safe_div(df['cash'], df['totalAssets'])
df['recv_to_assets'] = safe_div(df['netReceivables'], df['totalAssets'])
df['inv_to_assets'] = safe_div(df['inventory'], df['totalAssets'])
df['ppe_to_assets'] = safe_div(df['propertyPlantEquipment'], df['totalAssets'])
df['re_to_assets'] = safe_div(df['retainedEarnings'], df['totalAssets'])

# Sort by stock and year to calculate growth
df['endDate'] = pd.to_datetime(df['endDate'], errors='coerce')
df['year'] = df['endDate'].dt.year
df = df.sort_values(['stock', 'year'])

# Asset growth (year-over-year per firm)
df['asset_growth'] = df.groupby('stock')['totalAssets'].pct_change()

# Check results
df[['stock','year','leverage','equity_to_assets','current_ratio','asset_growth']].head(10)

/var/folders/_q/cys1ymsx41xf9dnnx6y_z3wc0000gn/T/ipykernel_4071/469313165.py:25: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['asset_growth'] = df.groupby('stock')['totalAssets'].pct_change()


,stock,year,leverage,equity_to_assets,current_ratio,asset_growth
3433,A,2016,0.455222,0.544393,3.846561,NaN
3432,A,2017,0.426181,0.573344,3.300871,0.081088
3431,A,2018,0.464817,0.534715,3.286080,0.013648
3430,A,2019,0.497672,0.502328,1.533173,0.106662
6209,AA,2016,0.540231,0.337734,1.127614,NaN
6208,AA,2017,0.610363,0.259242,1.303198,0.042172
6207,AA,2018,0.529631,0.348252,1.482700,-0.075371
6206,AA,2019,0.597704,0.281047,1.377292,-0.093045
10672,AAL,2016,0.926181,0.073819,0.744233,NaN
10671,AAL,2017,1.014777,-0.014777,0.595520,0.029469


In [7]:
df['asset_growth'] = df.groupby('stock')['totalAssets'].pct_change(fill_method=None)

Summary Statistics

In [8]:
ratios = ['leverage','equity_to_assets','current_ratio','quick_ratio',
          'wc_to_assets','cash_to_assets','recv_to_assets',
          'inv_to_assets','ppe_to_assets','re_to_assets','asset_growth']

summary = df[ratios].describe(percentiles=[0.01,0.05,0.25,0.5,0.75,0.95,0.99])
summary

,leverage,equity_to_assets,current_ratio,quick_ratio,wc_to_assets,cash_to_assets,recv_to_assets,inv_to_assets,ppe_to_assets,re_to_assets,asset_growth
count,17315.000000,17339.000000,17305.000000,17316.000000,17305.000000,1.692900e+04,14544.000000,9640.000000,15768.000000,16076.000000,12961.000000
mean,0.871319,0.019455,4.514191,2.500361,-0.147118,1.641971e-01,0.117999,0.110156,0.236378,-1.538740,42.581664
std,27.690612,27.731530,73.883281,14.340187,27.694148,2.220166e-01,0.134769,0.131756,0.275007,22.521300,4308.442775
min,0.000391,-3604.731707,0.000277,0.000000,-3604.731707,9.564882e-09,0.000001,0.000003,0.000007,-2293.695122,-0.951343
1%,0.017042,-2.489655,0.025990,0.010793,-0.864855,3.719568e-04,0.000731,0.000064,0.000973,-23.604824,-0.537774
5%,0.102214,-0.259357,0.059886,0.030813,-0.791558,3.218714e-03,0.004136,0.000470,0.004641,-6.411711,-0.252862
25%,0.353801,0.169204,0.951008,0.493800,-0.006425,2.254806e-02,0.027010,0.010783,0.029381,-0.605916,-0.021817
50%,0.560314,0.404006,1.734745,1.055496,0.122942,7.293275e-02,0.080083,0.068203,0.111300,0.021850,0.057637
75%,0.783372,0.623899,3.247870,1.963385,0.367928,2.012951e-01,0.158504,0.157789,0.350359,0.219092,0.204796
95%,1.037811,0.889094,11.260169,7.400847,0.807012,7.282148e-01,0.366612,0.370648,0.856504,0.675708,1.208680


Correlation Matrix